# Random Bidding Strategy

Date: 14/04/2017

This file implements a random bidding strategy for real time bidding and outputs validation and test bids.

In [1]:
import pandas as pd
import numpy as np

In [2]:
val = pd.read_csv('data/validation.csv')
test = pd.read_csv('data/test.csv')

In [3]:
# Function for creating columns with strategy bids
def apply_strategy(df, params, test=False):
    col_name = 'strategy_random_bid'
    col_name_validate = 'strategy_random_validate'
    if col_name not in df.keys():
        df[col_name] = int(0)
    if col_name_validate not in df.keys():
        df[col_name_validate] = int(0)
        
    df[col_name] = np.random.choice(range(0, params), df.shape[0])
     
    if test is False:
        df[col_name_validate] = df[col_name] - df['payprice']
    
    return df

# Function for validating strategy with different parameters
def validate_strategy(df, params):
    col_name = 'strategy_random_bid'
    col_name_validate = 'strategy_random_validate'
    
    df_temp = df[df[col_name_validate] > 0]
    
    cash_in_bank = glob_cash_in_bank    
    strategy_impressions = 0
    strategy_clicks = 0
    n_rows_in_budget = 0
    

    for row in df_temp.iterrows():
        row = row[1]
        if cash_in_bank > 0:
            n_rows_in_budget += 1
            if row[col_name] > row['payprice']:
                strategy_impressions += 1
                strategy_clicks += int(row['click'])
                cash_in_bank -= row['payprice']
        else:
            break
    if strategy_impressions>0:        
        strategy_CTR = strategy_clicks*100/strategy_impressions
    else:
        strategy_CTR = 0.0
    if strategy_clicks>0:
        strategy_CPC = (glob_cash_in_bank - cash_in_bank)/strategy_clicks
    else:
        strategy_CPC = 'Inf'

    dfr_index = len(df_results)
    df_results.loc[dfr_index] = ['random', params, strategy_impressions, strategy_clicks, cash_in_bank, strategy_CTR, strategy_CPC]

In [4]:
glob_cash_in_bank = 6250000 #6250
glob_results = dict()
columns = ['strategy_name', 'strategy_params', 'strategy_impressions', 'strategy_clicks', 'cash_in_bank', 
           'strategy_CTR', 'strategy_CPC']
#del(df_results)
df_results = pd.DataFrame(columns = columns)

In [9]:
# Now we optimize the parameter for constant bidding
param_range = np.arange(40, 500, 20)

for i, c in enumerate(param_range):
    validate_strategy(apply_strategy(val, c),  c)
    print("C: {} | Impressions: {} | Clicks: {} | Cash in Bank: {:.2f}$ | CTR: {:.2f}% | CPC: {:.2f}$".format(c, 
                                                                                                              float(df_results.tail(1).strategy_impressions), 
                                                                                                              float(df_results.tail(1).strategy_clicks), 
                                                                                                              float(df_results.tail(1).cash_in_bank), 
                                                                                                              float(df_results.tail(1).strategy_CTR), 
                                                                                                              float(df_results.tail(1).strategy_CPC)))

C: 40 | Impressions: 38480.0 | Clicks: 14.0 | Cash in Bank: 5654389.00$ | CTR: 0.04% | CPC: 42543.64$
C: 60 | Impressions: 60969.0 | Clicks: 15.0 | Cash in Bank: 4921631.00$ | CTR: 0.02% | CPC: 88557.93$
C: 80 | Impressions: 84812.0 | Clicks: 30.0 | Cash in Bank: 3814171.00$ | CTR: 0.04% | CPC: 81194.30$
C: 100 | Impressions: 110423.0 | Clicks: 51.0 | Cash in Bank: 2297937.00$ | CTR: 0.05% | CPC: 77491.43$
C: 120 | Impressions: 129993.0 | Clicks: 57.0 | Cash in Bank: 996980.00$ | CTR: 0.04% | CPC: 92158.25$
C: 140 | Impressions: 142564.0 | Clicks: 71.0 | Cash in Bank: -26.00$ | CTR: 0.05% | CPC: 88028.54$
C: 160 | Impressions: 133422.0 | Clicks: 75.0 | Cash in Bank: -11.00$ | CTR: 0.06% | CPC: 83333.48$
C: 180 | Impressions: 126171.0 | Clicks: 74.0 | Cash in Bank: -23.00$ | CTR: 0.06% | CPC: 84459.77$
C: 200 | Impressions: 119944.0 | Clicks: 76.0 | Cash in Bank: -5.00$ | CTR: 0.06% | CPC: 82236.91$
C: 220 | Impressions: 114912.0 | Clicks: 75.0 | Cash in Bank: -23.00$ | CTR: 0.07% | CPC

From the results above, it seems that the bid price of 180 provides the best results, because of the highest CTR and lowest CPC with still a high number of clicks and impressions. 

In [5]:
new_val = apply_strategy(val, 240, test=True)
new_test = apply_strategy(test, 240, test=True)

In [6]:
new_val[['bidid', 'strategy_random_bid']].to_csv('random_validation_bids.csv', encoding='utf-8')
new_test[['bidid', 'strategy_random_bid']].to_csv('random_testing_bids.csv', encoding='utf-8')